In [1]:
'''
    This program packages the labels and images 
from the underwater dataset and converts them 
to a .npz numpy array file.

    Thanks, Thomas, for labeling all of those images.
        -shadySource
'''
import os
import sys
import json
import PIL.Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np

debug = True #only load 1 images
shuffle = True # shuffle dataset

# label_dict = {"dog":0}
sheet_path='detected_sheet_total.csv' # bboxes记录
data_path='../data_detected_total/' # 图片数据

bbox_sheet=pd.read_csv(sheet_path,header=None,names=['file_name','bboxes'])
bbox_sheet['bboxes']=bbox_sheet['bboxes'].apply(eval)
bbox_sheet

,file_name,bboxes
0,dog.1.jpg,"[(20, 28, 251, 471, 16, dog, 0.83146054)]"
1,dog.1000.jpg,"[(119, 79, 165, 169, 16, dog, 0.92074239)]"
2,dog.10003.jpg,"[(41, 0, 313, 331, 16, dog, 0.83723885)]"
3,dog.10006.jpg,"[(0, 0, 499, 339, 16, dog, 0.87136871)]"
4,dog.10009.jpg,"[(187, 45, 106, 215, 16, dog, 0.86595523)]"
5,dog.10013.jpg,"[(107, 78, 295, 291, 16, dog, 0.80569953)]"
6,dog.10016.jpg,"[(5, 70, 336, 413, 16, dog, 0.83241522)]"
7,dog.10025.jpg,"[(123, 23, 196, 212, 16, dog, 0.83438087)]"
8,dog.10028.jpg,"[(106, 91, 277, 265, 16, dog, 0.92150164)]"
9,dog.10030.jpg,"[(153, 8, 95, 206, 16, dog, 0.83009422)]"


In [2]:
images_names = []
image_labels=[]
for i in tqdm(range(0,bbox_sheet.shape[0])):
    row=bbox_sheet.iloc[i]
    file_name,bboxes=row[0],row[1]
    for b in bboxes:
        img = np.array(PIL.Image.open(os.path.join(data_path,file_name)), dtype=np.uint8)
        w,h,d = img.shape
        if w<=0 or h<=0 or d<=0:
            continue
        x_min=b[0]
        y_min=b[1]
        x_max=x_min+b[2]
        y_max=y_min+b[3]
        images_names.append(file_name)
        image_labels.append([0,x_min,y_min,x_max,y_max]) 

# convert to numpy for saving       
images_names = np.array(images_names)
image_labels = np.array(image_labels)

100%|██████████| 10063/10063 [01:47<00:00, 93.80it/s] 


In [3]:
# shuffle dataset
if shuffle:
    np.random.seed(101)
    indices = np.arange(len(images_names))
    np.random.shuffle(indices)
    images_names, image_labels = images_names[indices], image_labels[indices]

In [4]:
print("Dataset contains {} images".format(images_names.shape[0]))
#save dataset
np.savez("detected_dogs", names=images_names, boxes=image_labels)
print('Data saved: detected_dogs.npz')

Dataset contains 10612 images
Data saved: detected_dogs.npz
